In [1]:
import numpy as np
import pandas as pd
import string 
from string import digits
import re
import nltk
from nltk.corpus import stopwords
from sklearn import preprocessing
import itertools

In [2]:
import scipy
import csv
from nltk.tokenize import word_tokenize
import re, csv, yaml
from nltk.corpus import wordnet as wn
from nltk.metrics import edit_distance
nltk.download('punkt')
import itertools
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\praga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\praga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [126]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [3]:
imfile = pd.read_csv("AIT624-imgdataset.csv")

In [4]:
imfile.dropna(axis=0,how="all",inplace = True)

In [6]:
imfile['Concept'].head(5)

0     [business=1, ceremony=1, festival=1, group=1,...
1     [mountain=6, outdoors=6, travel=6, tree=6, fo...
2     [people=4, city=3, politics=3, architecture=2...
3     [people=28, politics=18, protest=13, travel=1...
4     [automobile=1, business=1, police=1, transpor...
Name: Concept, dtype: object

In [7]:
def convert_dict(s):
    return re.sub(r'\W',' ',s)

In [8]:
imfile['list'] = imfile['Concept'].apply(convert_dict)

In [9]:
imfile['list'].head(5)

0      business 1  ceremony 1  festival 1  group 1 ...
1      mountain 6  outdoors 6  travel 6  tree 6  fo...
2      people 4  city 3  politics 3  architecture 2...
3      people 28  politics 18  protest 13  travel 1...
4      automobile 1  business 1  police 1  transpor...
Name: list, dtype: object

In [10]:
def remove_num(s):
    s = re.sub("\d+"," ", s) 
    return " ".join(s.split())

In [11]:
imfile['list'] = imfile['list'].apply(remove_num)

In [12]:
##
imfile['list'].head(5)

0    business ceremony festival group indoors peopl...
1    mountain outdoors travel tree forest landscape...
2    people city politics architecture building con...
3    people politics protest travel adult nobody ou...
4    automobile business police transportation vehicle
Name: list, dtype: object

In [13]:
word_tokenize(imfile['list'][0])

['business',
 'ceremony',
 'festival',
 'group',
 'indoors',
 'people',
 'performance',
 'religion',
 'restaurant']

In [14]:
imfile['list'].tail(20)

372    adult cabinet facial expression furniture girl...
373    people politics travel coast group outdoors ac...
374    road people action street transportation trave...
375                                                     
376    city travel people architecture street politic...
377    people architecture travel building outdoors a...
378             adult clothing one people portrait women
379    nobody architecture garden house luxury modern...
380    beach coast leisure recreation relaxation reso...
381    architecture building business city cityscape ...
382    travel architecture business nobody building p...
383    adult architecture building city competition c...
384    adult bar beverage cafe commerce counter desk ...
385    carpet chair dining table easy chair fireplace...
386    people politics adult men police protest road ...
387    people recreation swimming water swimming pool...
388    adult people outdoors women portrait politics ...
389    dwelling furniture house

In [133]:
imfile

,cluster_size,Date,Interval,Distance,File,Category,Concept,Event,list
0,1,Sun Jun 14 11:25:19 PDT 2009,0 Min,0.00 Miles,[DSCN0657],[Office=1],"[business=1, ceremony=1, festival=1, group=1,...",[],business ceremony festival group indoors peopl...
1,6,Sun Oct 18 13:40:54 CST 2009,4 Min,0.04 Miles,"[DSCN4196, DSCN4197, DSCN4198, DSCN4199, DSCN...","[Scenic Lookout=4, Hotel=1]","[mountain=6, outdoors=6, travel=6, tree=6, fo...",[URBAN_WALK],mountain outdoors travel tree forest landscape...
2,5,Sun Sep 20 14:23:57 CST 2009,12 Min,0.20 Miles,"[DSCN3925, DSCN3926, DSCN3927, DSCN3928, DSCN...","[Chinese Restaurant=2, Coffee Shop=2, Bus Sta...","[people=4, city=3, politics=3, architecture=2...",[URBAN_WALK],people city politics architecture building con...
3,44,Sun Sep 06 17:48:48 SGT 2009,63 Min,0.26 Miles,"[DSCN3382, DSCN3383, DSCN3384, DSCN3385, DSCN...","[Beach=8, Cable Car=7, Park=6, Event Space=3,...","[people=28, politics=18, protest=13, travel=1...",[URBAN_WALK],people politics protest travel adult nobody ou...
4,1,Sun Jun 07 14:37:40 PDT 2009,0 Min,0.00 Miles,[DSCN0381],[Bed & Breakfast=1],"[automobile=1, business=1, police=1, transpor...",[URBAN_WALK],automobile business police transportation vehicle
5,1,Wed Jun 17 13:15:04 MDT 2009,0 Min,0.00 Miles,[DSCN0876],[Bar=1],"[aircraft=1, airplane=1, flight=1, sky=1]",[],aircraft airplane flight sky
6,8,Tue Jun 16 12:15:33 MDT 2009,15 Min,0.01 Miles,"[DSCN0744, DSCN0747, DSCN0748, DSCN0751, DSCN...",[Pool=8],"[people=6, leisure=5, pool=5, poolside=5, rec...",[URBAN_WALK],people leisure pool poolside recreation swimme...
7,1,Sat Mar 06 15:15:42 IST 2010,0 Min,0.00 Miles,[DSCN6465],[Restaurant=1],"[asphalt=1, automobile=1, car=1, pavement=1, ...",[URBAN_WALK],asphalt automobile car pavement transportation...
8,1,Sun Oct 18 17:44:27 CST 2009,0 Min,0.00 Miles,[DSCN4246],[],"[cheese=1, dish=1, food=1, lunch=1, meal=1, n...",[DINNER],cheese dish food lunch meal nobody plate potat...
9,3,Sun Apr 18 15:13:04 PDT 2010,22 Min,0.34 Miles,"[DSCN7439, DSCN7441, DSCN7442]","[Bus Line=1, Sculpture Garden=1]","[asphalt=2, daytime=2, nobody=2, outdoors=2, ...",[URBAN_WALK],asphalt daytime nobody outdoors road street tr...


In [27]:
n=392
count=0
newlist = []
while(count<n):
    tokens = word_tokenize(imfile['list'][count])
    for a in range(0, len(tokens)-2):
        for b in range(a+1, len(tokens)-1):
            #print(a)
            #print(b)
            x = wn.synsets(tokens[a])
            y = wn.synsets(tokens[b])
            #print(x)
            #print(y)
            if (len(x) != 0) and (len(y) != 0):            
                ai = x[0]
                bi = y[0]
                xyz = ai.wup_similarity(bi)
                if xyz is None:
                    xyz = 0
                if xyz > 0.5:
                    tokens[b] = tokens[a]
                    #tokens = list(set(tokens))
                    #len(tokens)
                #print(ai)
                #print(bi)
                #print(xyz)
    #print(tokens)
    newlist.append(tokens)
    #print(newlist)
    #print(len(tokens))
    count+=1

# Lemmatisation

In [16]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [17]:
port = PorterStemmer()
wordnet_lemma = WordNetLemmatizer()

In [18]:
#wordnet_lemmatizer.lemmatize("dogs")

In [19]:
def normalize(s):
    stem = [port.stem(i) for i in s]
    lem = [wordnet_lemma.lemmatize(i,pos ='n') for i in stem]
    return ",".join([wordnet_lemma.lemmatize(i,pos ='v') for i in lem])
    #return lem   

In [20]:
imfile['list'] = imfile['list'].apply(normalize)
#imfile['list'] = imfile['list'].apply(split)

In [21]:
imfile['list'].head(20)

0     'business','ceremony','festival','group','indo...
1     'mountain','outdoors','travel','tree','forest'...
2     'people','city','politics','architecture','bui...
3     'people','politics','protest','travel','adult'...
4     'automobile','business','police','transportati...
5                  'aircraft','airplane','flight','sky'
6     'people','leisure','pool','poolside','recreati...
7     'asphalt','automobile','car','pavement','trans...
8     'cheese','dish','food','lunch','meal','nobody'...
9     'asphalt','daytime','nobody','outdoors','road'...
10    'people','adult','portrait','clothing','facial...
11    'road','action','army','auto','racing','automo...
12    'chair','contemporary','dwelling','floor','fur...
13    'adult','clothing','eyeglasses','eyewear','fac...
14    'outdoors','travel','street','architecture','c...
15    'adult','chair','clothing','education','furnit...
16    'architecture','city','clothing','group','peop...
17    'street','city','market','night','architec

# Since Stemming and Lemmentization doesn't give proper result So we do LSA

# LSA using Sci-Kit Learn

In [22]:
#concept = imfile['Concept'].str.cat(sep=', ') #merging the columns
#concept = concept.replace("\\", "\\\\") #converting to raw document
#concept

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer


vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,min_df= 1, stop_words='english',use_idf=True, smooth_idf = True,analyzer='word')
X_train_tfidf = vectorizer.fit_transform(imfile['list'])
X_train_tfidf[0]
print(X_train_tfidf[0])

  (0, 130)	0.256774609824
  (0, 169)	0.485432903405
  (0, 372)	0.306557576479
  (0, 452)	0.237372020944
  (0, 501)	0.262373563704
  (0, 679)	0.485432903405
  (0, 763)	0.305205969705
  (0, 772)	0.387895601206


In [24]:
X_train_tfidf

<392x1067 sparse matrix of type '<class 'numpy.float64'>'
	with 12309 stored elements in Compressed Sparse Row format>

In [25]:
print(X_train_tfidf)

  (0, 130)	0.256774609824
  (0, 169)	0.485432903405
  (0, 372)	0.306557576479
  (0, 452)	0.237372020944
  (0, 501)	0.262373563704
  (0, 679)	0.485432903405
  (0, 763)	0.305205969705
  (0, 772)	0.387895601206
  (1, 616)	0.232480202084
  (1, 980)	0.126575320627
  (1, 407)	0.206415586555
  (1, 530)	0.134504226269
  (1, 813)	0.224826194819
  (1, 782)	0.247917688028
  (1, 269)	0.113307529466
  (1, 395)	0.220218162025
  (1, 622)	0.171417922839
  (1, 855)	0.136115306676
  (1, 599)	0.268772508151
  (1, 997)	0.285997436562
  (1, 48)	0.285997436562
  (1, 477)	0.301044856788
  (1, 488)	0.15155279262
  (1, 656)	0.285997436562
  (1, 974)	0.376053884162
  :	:
  (391, 396)	0.107348873155
  (391, 933)	0.115748294477
  (391, 414)	0.121707778746
  (391, 208)	0.121707778746
  (391, 939)	0.121707778746
  (391, 13)	0.121707778746
  (391, 136)	0.121707778746
  (391, 833)	0.121707778746
  (391, 917)	0.121707778746
  (391, 959)	0.121707778746
  (391, 674)	0.130107200068
  (391, 1066)	0.130107200068
  (391, 36

In [32]:
lsa = TruncatedSVD(n_components =393)
X_lsa = lsa.fit(X_train_tfidf)
X_lsa

TruncatedSVD(algorithm='randomized', n_components=393, n_iter=5,
       random_state=None, tol=0.0)

In [33]:
lsa.components_

array([[ 0.00133409,  0.01223909,  0.00122176, ...,  0.00826164,
         0.00539557,  0.00097392],
       [-0.00127506, -0.00126453,  0.00642366, ...,  0.02485697,
        -0.00129329,  0.0007609 ],
       [ 0.00289255,  0.01014152, -0.00132307, ..., -0.00765521,
         0.00468253,  0.00317075],
       ..., 
       [ 0.00486597, -0.02425703,  0.0032891 , ...,  0.04744442,
         0.00771466, -0.00170934],
       [-0.00121957, -0.04690714, -0.00879716, ...,  0.01761861,
        -0.00872048,  0.00337087],
       [ 0.01654213, -0.00573071, -0.03438837, ..., -0.00646766,
        -0.02040382,  0.00063886]])

# Using WordNet - Synsets

In [48]:
from nltk.corpus import wordnet as wn

In [23]:
#synset
#imfile['New'] = np.empty((len(imfile), 0)).tolist()
#Checking for single word
def sm(lines):
    #for line in lines:
    for token in lines:
        syn_sets = wn.synsets(token)
            #syns = wordnet.synsets("business")
        return syn_sets[0].lemmas()[0].name()

In [24]:
imfile['New'] = imfile['list'].apply(sm)

IndexError: list index out of range

In [17]:
syn_sets = wn.synsets('Playing')
syn_sets.definition

AttributeError: 'list' object has no attribute 'definition'

In [29]:
for synset in (wn.synsets('Playing')):
        print(synset)
        nyms = ['hypernyms', 'hyponyms', 'meronyms', 'holonyms', 'sisterm_terms', 'troponyms', 'inherited_hypernyms']
        for i in nyms:
            try:
                print(getattr(synset, i)())
            except AttributeError as e: 
                print(e)
                pass

Synset('playing.n.01')
[Synset('musical_performance.n.01')]
[Synset('bowing.n.02'), Synset('piping.n.03'), Synset('stopping.n.02'), Synset('transposition.n.07')]
'Synset' object has no attribute 'meronyms'
'Synset' object has no attribute 'holonyms'
'Synset' object has no attribute 'sisterm_terms'
'Synset' object has no attribute 'troponyms'
'Synset' object has no attribute 'inherited_hypernyms'
Synset('playing.n.02')
[Synset('action.n.01')]
[Synset('bowling.n.03'), Synset('catching.n.01'), Synset('golfing.n.01'), Synset('pitching.n.01')]
'Synset' object has no attribute 'meronyms'
'Synset' object has no attribute 'holonyms'
'Synset' object has no attribute 'sisterm_terms'
'Synset' object has no attribute 'troponyms'
'Synset' object has no attribute 'inherited_hypernyms'
Synset('acting.n.01')
[Synset('activity.n.01'), Synset('performing_arts.n.01')]
[Synset('business.n.09'), Synset('hamming.n.01'), Synset('heroics.n.01'), Synset('impersonation.n.03'), Synset('method_acting.n.01'), Syns

In [139]:
def replaceit(data,b,a,num):
    return data.replace(b,a,num)    

In [156]:
for a, b in itertools.combinations(abc, 2):
    ai = wn.synsets(a)[0]
    bi = wn.synsets(b)[0]
    print(a)
    print(b)
    xyz = ai.wup_similarity(bi)
    if xyz is None:
        xyz = 0
    if xyz > 0.5:
        b = a
        #print(abc.replace(b,a))
    print(abc)

business
ceremony
['business', 'ceremony', 'festival', 'group', 'indoors', 'people', 'performance', 'religion', 'restaurant']
business
festival
['business', 'ceremony', 'festival', 'group', 'indoors', 'people', 'performance', 'religion', 'restaurant']
business
group
['business', 'ceremony', 'festival', 'group', 'indoors', 'people', 'performance', 'religion', 'restaurant']
business
indoors
['business', 'ceremony', 'festival', 'group', 'indoors', 'people', 'performance', 'religion', 'restaurant']
business
people
['business', 'ceremony', 'festival', 'group', 'indoors', 'people', 'performance', 'religion', 'restaurant']
business
performance
['business', 'ceremony', 'festival', 'group', 'indoors', 'people', 'performance', 'religion', 'restaurant']
business
religion
['business', 'ceremony', 'festival', 'group', 'indoors', 'people', 'performance', 'religion', 'restaurant']
business
restaurant
['business', 'ceremony', 'festival', 'group', 'indoors', 'people', 'performance', 'religion', 'restau

In [45]:
def syn(word, lch_threshold=2.26):
    for net1 in wn.synsets(word):
        for net2 in wn.all_synsets():
            try:
                lch = net1.lch_similarity(net2)
            except:
                continue
            # The value to compare the LCH to was found empirically.
            # (The value is very application dependent. Experiment!)
            if lch >= 0.5:
                yield (net1, net2, lch)

In [16]:
def sim(word1, word2, lch_threshold=2.15, verbose=False):
    """Determine if two (already lemmatized) words are similar or not.

    Call with verbose=True to print the WordNet senses from each word
    that are considered similar.

    The documentation for the NLTK WordNet Interface is available here:
    http://nltk.googlecode.com/svn/trunk/doc/howto/wordnet.html
    """
    from nltk.corpus import wordnet as wn
    results = []
    for net1 in wn.synsets(word1):
        for net2 in wn.synsets(word2):
            try:
                lch = net1.lch_similarity(net2)
            except:
                continue
            # The value to compare the LCH to was found empirically.
            # (The value is very application dependent. Experiment!)
            if lch >= lch_threshold:
                results.append((net1, net2))
    if not results:
        return False
    if verbose:
        for net1, net2 in results:
            print(net1)
            print(net1.definition)
            print(net2)
            print(net2.definition)
            print('path similarity:')
            print(net1.path_similarity(net2))
            print('lch similarity:')
            print(net1.lch_similarity(net2))
            print('wup similarity:')
            print(net1.wup_similarity(net2))
            print('-' * 79)
    return True

In [17]:
for a, b in itertools.combinations(abc, 2):
    sim(a, b, verbose= False)
    

NameError: name 'abc' is not defined

In [18]:
def lange(abc):
    for a, b in itertools.combinations(abc, 2):
        ai = wn.synsets(a)[0]
        bi = wn.synsets(b)[0]
        print(ai)
        print(bi)
        xyz = ai.wup_similarity(bi)
        if xyz is None:
            xyz = 0
        xyz = [float(xyz)]
        print(xyz)

In [19]:
rc = pd.read_csv('RedCon.csv',delimiter= ',')

In [20]:
rc.head()

,Concept_wup,Concept_lch
0,"[business, ceremony, festival, business, indoo...","[business, ceremony, festival, business, indoo..."
1,"[mountain, outdoors, travel, tree, forest, lan...","[mountain, outdoors, travel, tree, forest, lan..."
2,"[people, city, politics, architecture, archite...","[people, city, politics, architecture, archite..."
3,"[people, politics, protest, protest, adult, ad...","[people, politics, protest, travel, adult, adu..."
4,"[automobile, business, business, automobile, v...","[automobile, business, business, transportatio..."


In [21]:
def lenth(line):
    abc = word_tokenize(line)
    abc =  list(set(abc))
    return len(abc)

In [22]:
rc['Concept_wup'] = rc['Concept_wup'].apply(convert_dict)
rc['Concept_lch'] = rc['Concept_lch'].apply(convert_dict)

In [23]:
rc['concept'] = imfile['list'].apply(lenth)
rc['wup_len'] = rc['Concept_wup'].apply(lenth)
rc['lch_len'] = rc['Concept_lch'].apply(lenth)
rc['Concepts'] = imfile['Concept']

In [233]:
pd.set_option('display.max_colwidth', -1)

In [24]:
rc

,Concept_wup,Concept_lch,concept,wup_len,lch_len,Concepts
0,business ceremony festival business indoo...,business ceremony festival business indoo...,9,6,7,"[business=1, ceremony=1, festival=1, group=1,..."
1,mountain outdoors travel tree forest lan...,mountain outdoors travel tree forest lan...,22,14,18,"[mountain=6, outdoors=6, travel=6, tree=6, fo..."
2,people city politics architecture archite...,people city politics architecture archite...,37,15,27,"[people=4, city=3, politics=3, architecture=2..."
3,people politics protest protest adult ad...,people politics protest travel adult adu...,110,26,51,"[people=28, politics=18, protest=13, travel=1..."
4,automobile business business automobile v...,automobile business business transportatio...,5,3,4,"[automobile=1, business=1, police=1, transpor..."
5,aircraft aircraft flight sky,aircraft aircraft flight sky,4,3,3,"[aircraft=1, airplane=1, flight=1, sky=1]"
6,people leisure pool poolside recreation ...,people leisure pool poolside recreation ...,44,20,32,"[people=6, leisure=5, pool=5, poolside=5, rec..."
7,asphalt asphalt asphalt asphalt asphalt ...,asphalt automobile automobile pavement as...,7,2,4,"[asphalt=1, automobile=1, car=1, pavement=1, ..."
8,cheese dish cheese cheese cheese nobody ...,cheese dish cheese lunch lunch nobody p...,13,4,7,"[cheese=1, dish=1, food=1, lunch=1, meal=1, n..."
9,asphalt daytime nobody outdoors asphalt ...,asphalt daytime nobody outdoors asphalt ...,26,11,17,"[asphalt=2, daytime=2, nobody=2, outdoors=2, ..."
